In [48]:
import pandas as pd
import numpy as np
#import seaborn
import sklearn as skl
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
#import future
#from autograd import grad
%matplotlib inline

In [49]:
demand_pred=pd.read_csv('demand_pred.csv')
demand_act=pd.read_csv('demand.csv')
price_pred=pd.read_csv('price_pred.csv')
price_act=pd.read_csv('price.csv')
solar_pred=pd.read_csv('solar_pred.csv')
solar_act=pd.read_csv('solar.csv')
lb_demand=pd.read_csv('Data_TestSet_PrivateEvaluation/Demand_Test_pred.csv', header=None)
lb_price=pd.read_csv('Data_TestSet_PrivateEvaluation/Price_Test_pred.csv', header=None)
lb_solar=pd.read_csv('Data_TestSet_PrivateEvaluation/Solar_Test_pred.csv', header=None)

In [50]:
price_pred_train=price_pred[:-50]  
price_pred_test=price_pred[-50:]
price_act_train= price_act[:-50]
price_act_test=price_act[-50:]

solar_pred_train=solar_pred[:-50]
solar_pred_test=solar_pred[-50:]
solar_act_train =solar_act[:-50]
solar_act_test=solar_act[-50:]

demand_pred_train=demand_pred[:-50]
demand_pred_test=demand_pred[-50:]
demand_act_train=demand_act[:-50]
demand_act_test=demand_act[-50:]



In [51]:
#lin reg by passing all 24 value in a day as feature and outputting 24 values
est = linear_model.LinearRegression()

est.fit(demand_pred,demand_act)
demandL24=est.predict(lb_demand)

est.fit(price_pred,price_act)
priceL24=est.predict(lb_price)

est.fit(solar_pred,solar_act)
solarL24=est.predict(lb_solar)

In [52]:
type(lb_solar)

pandas.core.frame.DataFrame

In [53]:
from sklearn import neighbors

In [54]:
#KNN using 24 features at once

estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=7, n_jobs=-1)
estKnn24.fit(demand_pred, demand_act)
demandKnn24 = estKnn24.predict(lb_demand)

estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
estKnn24.fit(price_pred, price_act)
priceKnn24 = estKnn24.predict(lb_price)

estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=7, n_jobs=-1)
estKnn24.fit(solar_pred, solar_act)
solarKnn24 = estKnn24.predict(lb_solar)

print("done4")
#score2(pd.DataFrame(demandKnn24), pd.DataFrame(priceKnn24), pd.DataFrame(solarKnn24), np.concatenate( (demand_act_test,price_act_test,solar_act_test), axis=1) )

done4


In [55]:
#Elasticnet using 1 feature
estEN = linear_model.ElasticNetCV(cv=18)

#est.fit(np.array(demand_pred).reshape(-1,1), np.array(demand_act).ravel())
#scores=cross_val_score(estEN, np.array(demand_pred).reshape(-1,1), np.array(demand_act).ravel() )
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
estEN.fit( np.array(demand_pred).reshape(-1,1), np.array(demand_act).ravel() )
demandEN = estEN.predict(np.array(lb_demand).reshape(-1,1))

''' 
#scores=cross_val_score(estEN, np.array(price_pred).reshape(-1,1), np.array(price_act).ravel() )
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
estEN.fit( np.array(price_pred).reshape(-1,1), np.array(price_act).ravel())
priceEN = estEN.predict(np.array(lb_price).reshape(-1,1))

#scores=cross_val_score(estEN, np.array(solar_pred).reshape(-1,1), np.array(solar_act).ravel() )
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
estEN.fit( np.array(solar_pred).reshape(-1,1), np.array(solar_act).ravel()) 
solarEN = estEN.predict(np.array(lb_solar).reshape(-1,1))
''' 
print("done3")
#print(str(pd.DataFrame(demandEN).shape)+"s ")

#score3(pd.DataFrame(demandEN.reshape(50,24)), pd.DataFrame(priceEN.reshape(50,24)), pd.DataFrame(solarEN.reshape(50,24)), (demand_act_test), (price_act_test), (solar_act_test) )

done3


In [ ]:
0.96734275],[ 1.63682223]
0.96745217],[ 1.63512839
        
1.02092761],[ 1.64126239
 0.99368733],[ 1.6376168

In [56]:
estEN.set_params()

ElasticNetCV(alphas=None, copy_X=True, cv=18, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0)

In [57]:
''' 
def lin_reg(w,X):#w:2x1 X:nx1 X_new: nx2
    bias_feature=np.ones((X.shape[0], 1))
    X_new = np.concatenate((X, bias_feature), axis=1)
    return np.dot(w,X_new)

def custom_loss_given_weights(w):
    y_predicted = lin_reg(w, X)
    return score4(y_predicted, y)
    
gradient = grad(custom_loss_given_weights)
''' 
def lin_reg(w,X):#w:2x1 X:nx1 X_new: nx2
    bias_feature=np.ones((X.shape[0], 1))
    X_new = np.concatenate((X, bias_feature), axis=1)
    #print("X_new shape:"+str(X_new.shape))
    #print("bias_feature"+str(bias_feature.shape))
    #print("w shape:"+str(w.shape))
    return np.dot(X_new, w)

def custom_loss_given_weights(w):
    y_predicted = lin_reg(w, X)
    #print("within custom loss given wt X:"+str(X))
    return score4(y_predicted, y)
  
gradient = grad(custom_loss_given_weights)


NameError: name 'grad' is not defined

In [58]:
def score4(yhat, y):#yhat is array of demand, y is dataframe of demand act, price act, solar act #(bid, quantity,  demand_act, price_act, solar_act):
    
    bid = np.array(priceKnn24).reshape(-1,1)
    quantity = yhat-np.array(solarKnn24).reshape(-1,1)
    
    demand_act = np.array(y.iloc[:,:24]).reshape(-1,1)
    price_act = np.array(y.iloc[:,24:48]).reshape(-1,1)
    solar_act = np.array(y.iloc[:,48:72]).reshape(-1,1)
    
    #print("yhat:"+str(yhat))
    #print("y:"+str(y))
    #demand_act = demand_act_cv_test
    #price_act = price_act_cv_test
    #solar_act = solar_act_cv_test
    
    battery=0
    excess=0
    paisa=0
    for i in range(0, bid.shape[0]):
        
        #p-quantity we ordered
        #q-how much less we are ordering from actual
        #r-how much is actually required
        p = quantity[i]
        r = demand_act[i]-solar_act[i]
        q = r-p
        #print("q:"+str(q))
        if bid[i]>=price_act[i]: #bid won
            paisa=paisa+p*bid[i]
            if q>0:
                paisa=paisa+(q-min(q,4,battery))*7
                battery=battery-min(q,4,battery)
            else:
                battery=battery+min(-q,5)
                battery=min(battery,25)
        else:#bid lost
            if r>=0:
                paisa=paisa+(r-min(r,4,battery))*7
                battery=battery-min(r,4,battery)
            else:
                battery=battery+min(-r,5)
                battery=min(battery, 25)
                
    #print("battery left: "+str(battery) )
    #print("paisa:"+str(paisa))
    return paisa

In [59]:
def score5(yhat, y):#yhat is array of demand, y is dataframe of demand act, price act, solar act #(bid, quantity,  demand_act, price_act, solar_act):
    
    bid = np.array(priceKnn24).reshape(-1,1)
    quantity = yhat-np.array(solarKnn24).reshape(-1,1)
    
    demand_act = np.array(y.iloc[:,:24]).reshape(-1,1)
    price_act = np.array(y.iloc[:,24:48]).reshape(-1,1)
    solar_act = np.array(y.iloc[:,48:72]).reshape(-1,1)
    
    #print("yhat:"+str(yhat))
    #print("y:"+str(y))
    #demand_act = demand_act_cv_test
    #price_act = price_act_cv_test
    #solar_act = solar_act_cv_test
    bill_act = np.multiply(demand_act-solar_act, price_act)
    bill_ped = np.multiply(quantity, bid)
    
    cost=np.sum(np.sum(np.square(bill_act-bill_ped), axis=1), axis=0)
    return cost
    ''' 
    battery=0
    excess=0
    paisa=0
    for i in range(0, bid.shape[0]):
        
        #p-quantity we ordered
        #q-how much less we are ordering from actual
        #r-how much is actually required
        p = quantity[i]
        r = demand_act[i]-solar_act[i]
        q = r-p
        #print("q:"+str(q))
        if bid[i]>=price_act[i]: #bid won
            paisa=paisa+p*bid[i]
            if q>0:
                paisa=paisa+(q-min(q,4,battery))*7
                battery=battery-min(q,4,battery)
            else:
                battery=battery+min(-q,5)
                battery=min(battery,25)
        else:#bid lost
            if r>=0:
                paisa=paisa+(r-min(r,4,battery))*7
                battery=battery-min(r,4,battery)
            else:
                battery=battery+min(-r,5)
                battery=min(battery, 25)
                
    #print("battery left: "+str(battery) )
    #print("paisa:"+str(paisa))
    return paisa
    ''' 

In [60]:
my_scorer=make_scorer(score4, greater_is_better=False)

In [61]:
demandEN.shape

(2400,)

In [62]:
demand_pred_train.shape

(850, 24)

In [261]:
splits=17
cv = KFold(n_splits=splits, shuffle=True, random_state=1)
scores=0
score=0
scores_array = []
for p_neighbours in [4]:#[3,4,5,6,7,8,9]:
    
    for s_neighbours in [9]:
        
        scores=0
        for traincv, testcv in cv.split(demand_pred_train,demand_act_train):

            price_pred_cv_train = price_pred_train.iloc[traincv]
            price_pred_cv_test  = price_pred_train.iloc[testcv]
            price_act_cv_train  = price_act_train.iloc[traincv]
            price_act_cv_test   = price_act_train.iloc[testcv]

            demand_pred_cv_train = demand_pred_train.iloc[traincv]
            demand_pred_cv_test  = demand_pred_train.iloc[testcv]
            demand_act_cv_train  = demand_act_train.iloc[traincv]
            demand_act_cv_test   = demand_act_train.iloc[testcv]

            solar_pred_cv_train = solar_pred_train.iloc[traincv]
            solar_pred_cv_test  = solar_pred_train.iloc[testcv]
            solar_act_cv_train  = solar_act_train.iloc[traincv]
            solar_act_cv_test   = solar_act_train.iloc[testcv]

            #y = pd.concat((demand_act_cv_test, price_act_cv_test, solar_act_cv_test), axis=1, ignore_index=True)

            #Knn in the house
            #print("train shape:"+str(demand_pred_cv_train.shape))
            #print("test shape:"+str(demand_pred_cv_test.shape))

            estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=p_neighbours, n_jobs=-1)
            estKnn24.fit(price_pred_cv_train, price_act_cv_train)
            priceKnn24 = estKnn24.predict(price_pred_cv_test)+price_std2+0.34 #for cv 0.5 was faound earlier

            estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=s_neighbours, n_jobs=-1)
            estKnn24.fit(solar_pred_cv_train, solar_act_cv_train)
            solarKnn24 = estKnn24.predict(solar_pred_cv_test)+1

            estEN.fit( np.array(demand_pred_cv_train).reshape(-1,1), np.array(demand_act_cv_train).ravel() )
            demandEN = (estEN.predict(np.array(demand_pred_cv_test).reshape(-1,1))).reshape(-1,1)

            y = pd.concat((demand_pred_cv_test, price_pred_cv_test, solar_pred_cv_test), axis=1, ignore_index=True)
            
            score=score4(demandEN, y)
            #print(str(score))
            scores+=score


        print("p_neighbours:"+str(p_neighbours)+" s_neighbours:"+str(s_neighbours))
        print("scores: "+str(scores/splits))
        scores_array.append(scores/splits)

p_neighbours:4 s_neighbours:9
scores: [ 309124.17694396]


In [ ]:
price_std2

In [49]:
scores_array.sort()

In [ ]:
scores_array

In [13]:
from autograd import grad

In [ ]:
def lin_reg(w,X):#w:2x1 X:nx1 X_new: nx2
    bias_feature=np.ones((X.shape[0], 1))
    X_new = np.concatenate((X, bias_feature), axis=1)
    print("X_new shape:"+str(X_new.shape))
    print("bias_feature"+str(bias_feature.shape))
    print("w shape:"+str(w.shape))
    return np.dot(X_new, w)

def custom_loss_given_weights(w):
    y_predicted = lin_reg(w, X)
    return score4(y_predicted, y)
    
gradient = grad(custom_loss_given_weights)

X = np.array(demand_pred_test).reshape(-1,1)
y = pd.concat((demand_pred_test, price_pred_test, solar_pred_test), axis=1, ignore_index=True)
    
print("X shape:"+str(X.shape))
print("y shape:"+str(y.shape))

weights = np.zeros((X.shape[1]+1,1))

for i in range(10000):
    if i % 1000 == 0:
        print('Iteration %-4d | Loss: %.4f' % (i, custom_loss_given_weights(weights)))
    weights -= gradient(weights) * .01



In [46]:
np.ones((10, 1))

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [40]:
a=np.array([[1,1],[2,4]])
b=np.array([3,3]).reshape(-1,1)

In [41]:
b

array([[3],
       [3]])

In [42]:
a

array([[1, 1],
       [2, 4]])

In [43]:
(np.sum(np.multiply(b,a), axis=1)).shape

(2,)

In [44]:
np.dot(a,b)

array([[ 6],
       [18]])

In [63]:
price_std2=(price_act-price_pred)

In [64]:
price_std2[price_std2<0]=0

In [65]:
price_std2.describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.00000,900.000000,900.000000,900.000000,...,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,0.065211,0.059878,0.055989,0.060356,0.063667,0.077611,0.14220,0.149889,0.182711,0.174800,...,0.124622,0.106622,0.114544,0.123556,0.130967,0.107856,0.108722,0.100700,0.090811,0.057333
std,0.087030,0.083870,0.079243,0.080793,0.083514,0.111632,0.18886,0.218895,0.242818,0.236641,...,0.169541,0.151813,0.153537,0.159969,0.174595,0.151415,0.145109,0.137718,0.116470,0.079199
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01000,0.000000,0.015000,0.010000,...,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.120000,0.110000,0.100000,0.120000,0.120000,0.150000,0.26000,0.260000,0.340000,0.320000,...,0.220000,0.200000,0.210000,0.230000,0.260000,0.220000,0.210000,0.190000,0.180000,0.110000
max,0.330000,0.330000,0.340000,0.340000,0.360000,0.450000,0.72000,0.990000,1.050000,0.990000,...,0.730000,0.600000,0.600000,0.660000,0.780000,0.580000,0.540000,0.560000,0.430000,0.300000


In [66]:
price_std2=price_std2.describe().iloc[2,:]

In [67]:
price_std2=np.array(price_std2).reshape(1,-1)

In [68]:
price_std2.shape

(1, 24)

In [69]:
price_std2

array([[ 0.08703023,  0.08386974,  0.07924294,  0.08079338,  0.08351364,
         0.11163246,  0.18886007,  0.21889496,  0.24281805,  0.23664099,
         0.20785652,  0.16684295,  0.16782499,  0.15498694,  0.16954093,
         0.15181267,  0.15353714,  0.15996879,  0.17459522,  0.15141513,
         0.14510949,  0.13771801,  0.11646971,  0.07919927]])

In [122]:
priceslb_bk = np.copy(priceslb)

In [70]:
#submission

In [71]:
price_std=(price_act-price_pred).describe().iloc[2,:]
quantity_std=(demand_act-solar_act-(demand_pred-solar_pred))

In [72]:
price_pred.shape

(900, 24)

In [73]:
#KNN using 24 features at once

estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=7, n_jobs=-1)
estKnn24.fit(demand_pred, demand_act)
demandKnn24 = estKnn24.predict(lb_demand)

estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=4, n_jobs=-1)
estKnn24.fit(price_pred, price_act)
priceKnn24 = estKnn24.predict(lb_price)

estKnn24 = neighbors.KNeighborsRegressor(n_neighbors=9, n_jobs=-1)
estKnn24.fit(solar_pred, solar_act)
solarKnn24 = estKnn24.predict(lb_solar)

print("done4")
#score2(pd.DataFrame(demandKnn24), pd.DataFrame(priceKnn24), pd.DataFrame(solarKnn24), np.concatenate( (demand_act_test,price_act_test,solar_act_test), axis=1) )

done4


In [74]:
type(estEN.coef_)

numpy.ndarray

In [75]:
estEN.coef_

array([ 0.97760076])

In [76]:
estEN.intercept_

1.7105931282868312

In [77]:
#Elasticnet using 1 feature
estEN = linear_model.ElasticNetCV(cv=18)

#est.fit(np.array(demand_pred).reshape(-1,1), np.array(demand_act).ravel())
#scores=cross_val_score(estEN, np.array(demand_pred).reshape(-1,1), np.array(demand_act).ravel() )
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
estEN.fit( np.array(demand_pred).reshape(-1,1), np.array(demand_act).ravel() )
#estEN.coef_= np.array([1])
#estEN.intercept_= 1.6376168
demandEN = estEN.predict(np.array(lb_demand).reshape(-1,1))

''' 
#scores=cross_val_score(estEN, np.array(price_pred).reshape(-1,1), np.array(price_act).ravel() )
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
estEN.fit( np.array(price_pred).reshape(-1,1), np.array(price_act).ravel())
priceEN = estEN.predict(np.array(lb_price).reshape(-1,1))

#scores=cross_val_score(estEN, np.array(solar_pred).reshape(-1,1), np.array(solar_act).ravel() )
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
estEN.fit( np.array(solar_pred).reshape(-1,1), np.array(solar_act).ravel()) 
solarEN = estEN.predict(np.array(lb_solar).reshape(-1,1))
''' 
print("done3")
#print(str(pd.DataFrame(demandEN).shape)+"s ")

#score3(pd.DataFrame(demandEN.reshape(50,24)), pd.DataFrame(priceEN.reshape(50,24)), pd.DataFrame(solarEN.reshape(50,24)), (demand_act_test), (price_act_test), (solar_act_test) )

done3


In [ ]:
0.96734275],[ 1.63682223]
0.96745217],[ 1.63512839
        
1.02092761],[ 1.64126239
 0.99368733],[ 1.6376168

In [97]:
battery1200 = pd.read_csv('battery2400.csv', header=None)

In [98]:
battery1200.shape

(2400, 1)

In [80]:
demandslb = np.array([demandEN.reshape(100,24)])
priceslb = np.array([priceKnn24])
solarslb = np.array([solarKnn24])


In [81]:
# i-demand
#j-price
#k-solar
i=0
j=0
k=0

In [82]:
a=0.15 #on cv 0.5 worked
b=-1

In [83]:
priceslbx=[]
priceslbx=np.copy(priceslb[j])

In [84]:
priceslbx[:,0].shape

(100,)

In [85]:
np.sum(np.nan_to_num(np.multiply(demandslb[i][priceslbx>7], (priceslbx[priceslbx>7]-7) ) ) )

216.14545666154754

In [86]:
priceslbx

array([[ 1.21  ,  1.195 ,  1.105 , ...,  3.065 ,  2.335 ,  1.405 ],
       [ 1.7825,  1.7025,  1.655 , ...,  2.73  ,  2.0275,  1.6875],
       [ 1.3425,  1.3575,  1.3575, ...,  3.22  ,  2.1525,  1.345 ],
       ..., 
       [ 1.3125,  1.3225,  1.3125, ...,  2.33  ,  2.0625,  1.82  ],
       [ 1.8125,  1.705 ,  1.6425, ...,  2.615 ,  2.33  ,  1.815 ],
       [ 1.9175,  1.785 ,  1.6725, ...,  2.5   ,  2.535 ,  2.11  ]])

In [87]:
priceslbx=priceslbx+price_std2+0.34 #based on cv
priceslbx[priceslbx>=7]=6.85

#priceslbx[priceslbx>=7]=6.3
#priceslbx=priceslbx+a
#priceslbx[priceslbx>=7]=6.85

In [88]:
priceslbx

array([[ 1.63703023,  1.61886974,  1.52424294, ...,  3.54271801,
         2.79146971,  1.82419927],
       [ 2.20953023,  2.12636974,  2.07424294, ...,  3.20771801,
         2.48396971,  2.10669927],
       [ 1.76953023,  1.78136974,  1.77674294, ...,  3.69771801,
         2.60896971,  1.76419927],
       ..., 
       [ 1.73953023,  1.74636974,  1.73174294, ...,  2.80771801,
         2.51896971,  2.23919927],
       [ 2.23953023,  2.12886974,  2.06174294, ...,  3.09271801,
         2.78646971,  2.23419927],
       [ 2.34453023,  2.20886974,  2.09174294, ...,  2.97771801,
         2.99146971,  2.52919927]])

In [89]:
submission = np.concatenate( ( (priceslbx).reshape(-1,1), demandslb[i].reshape(-1,1)-solarslb[k].reshape(-1,1)+b)  , axis=1 )

In [90]:
submission.shape

(2400, 2)

In [91]:
submissiondf = pd.DataFrame(submission)

In [99]:
submissiondf.to_csv('24.csv', index=False, index_label=False, header=None)

In [93]:
fuck=np.array(lb_price).reshape(-1,1)

In [94]:
fuck[-5:]

array([[ 3.04],
       [ 2.56],
       [ 2.74],
       [ 2.34],
       [ 1.87]])

In [42]:
price_std2

array([[ 0.08703023,  0.08386974,  0.07924294,  0.08079338,  0.08351364,
         0.11163246,  0.18886007,  0.21889496,  0.24281805,  0.23664099,
         0.20785652,  0.16684295,  0.16782499,  0.15498694,  0.16954093,
         0.15181267,  0.15353714,  0.15996879,  0.17459522,  0.15141513,
         0.14510949,  0.13771801,  0.11646971,  0.07919927]])

In [ ]:
submissiondf = pd.read_csv('24.csv', header=None)
submissiondf

In [ ]:
charged=0
discharged=0
#a-charge by, b-discharge by  1.5 1
a=1.5
b=1
submissiondf = pd.read_csv('24.csv', header=None)

for c in range(len(battery1200)):
    if battery1200.iloc[c,0]==1:
        print("chege"+str(submissiondf.iloc[c,0]))
        submissiondf.iloc[c,1] = submissiondf.iloc[c,1] + a
        charged=charged+1
        continue
    if battery1200.iloc[c,0]==2 :
        print("discharge:"+str(submissiondf.iloc[c,0]))
        submissiondf.iloc[c,1] = submissiondf.iloc[c,1] - b
        discharged=discharged+1
        continue
submissiondf.to_csv('private_15.csv', index=False, index_label=False, header=None)

In [668]:
charged

467

In [ ]:
submissiondf

In [669]:
1200-467*2

266

In [671]:
submissiondf.shape

(1200, 2)

In [672]:
submissiondf.to_csv('23.csv', index=False, index_label=False)

In [123]:
submissiondf.iloc[:,0].shape

(1200,)